# Multilayer Perceptron

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum 4.3.10.0
%sql postgresql://gpdbchina@10.194.10.68:61000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum 4.2.3.0
#%sql postgresql://gpdbchina@10.194.10.68:55000/madlib

u'Connected: gpdbchina@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.12-dev, git revision: rel/v1.11-48-gff1b0f8, cmake configuration time: Tue Aug 15 00:36:04 UTC 2017, build type: Release, build system: Linux-2.6.18-238.27.1.el5.hotfix.bz516490, C compiler: gcc 4.4.0, C++ compiler: g++ 4.4.0"


# 1.  Create input table for classification

In [4]:
%%sql 
DROP TABLE IF EXISTS iris_data;

CREATE TABLE iris_data(
    id integer,
    attributes numeric[],
    class_text varchar,
    class integer
);

INSERT INTO iris_data VALUES
(1,ARRAY[5.1,3.5,1.4,0.2],'Iris-setosa',1),
(2,ARRAY[4.9,3.0,1.4,0.2],'Iris-setosa',1),
(3,ARRAY[4.7,3.2,1.3,0.2],'Iris-setosa',1),
(4,ARRAY[4.6,3.1,1.5,0.2],'Iris-setosa',1),
(5,ARRAY[5.0,3.6,1.4,0.2],'Iris-setosa',1),
(6,ARRAY[5.4,3.9,1.7,0.4],'Iris-setosa',1),
(7,ARRAY[4.6,3.4,1.4,0.3],'Iris-setosa',1),
(8,ARRAY[5.0,3.4,1.5,0.2],'Iris-setosa',1),
(9,ARRAY[4.4,2.9,1.4,0.2],'Iris-setosa',1),
(10,ARRAY[4.9,3.1,1.5,0.1],'Iris-setosa',1),
(11,ARRAY[7.0,3.2,4.7,1.4],'Iris-versicolor',2),
(12,ARRAY[6.4,3.2,4.5,1.5],'Iris-versicolor',2),
(13,ARRAY[6.9,3.1,4.9,1.5],'Iris-versicolor',2),
(14,ARRAY[5.5,2.3,4.0,1.3],'Iris-versicolor',2),
(15,ARRAY[6.5,2.8,4.6,1.5],'Iris-versicolor',2),
(16,ARRAY[5.7,2.8,4.5,1.3],'Iris-versicolor',2),
(17,ARRAY[6.3,3.3,4.7,1.6],'Iris-versicolor',2),
(18,ARRAY[4.9,2.4,3.3,1.0],'Iris-versicolor',2),
(19,ARRAY[6.6,2.9,4.6,1.3],'Iris-versicolor',2),
(20,ARRAY[5.2,2.7,3.9,1.4],'Iris-versicolor',2);

SELECT * FROM iris_data ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,attributes,class_text,class
1,"[Decimal('5.1'), Decimal('3.5'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
2,"[Decimal('4.9'), Decimal('3.0'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
3,"[Decimal('4.7'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris-setosa,1
4,"[Decimal('4.6'), Decimal('3.1'), Decimal('1.5'), Decimal('0.2')]",Iris-setosa,1
5,"[Decimal('5.0'), Decimal('3.6'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
6,"[Decimal('5.4'), Decimal('3.9'), Decimal('1.7'), Decimal('0.4')]",Iris-setosa,1
7,"[Decimal('4.6'), Decimal('3.4'), Decimal('1.4'), Decimal('0.3')]",Iris-setosa,1
8,"[Decimal('5.0'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris-setosa,1
9,"[Decimal('4.4'), Decimal('2.9'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
10,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris-setosa,1


#  2. Classification

Generate a multilayer perceptron with a single hidden layer of 5 units. Use the attributes column as the independent variables, and use the class column as the classification. Set the tolerance to 0 so that 500 iterations will be run. Use a hyperbolic tangent activation function. The model will be written to mlp_model.

In [5]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary;

-- Set seed so results are reproducible
SELECT setseed(0);
SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=500,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    TRUE              -- Verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


# 3.  View the classification model

In [6]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[0.134859643584, 0.200285896402, -0.281831690249, 0.755199357168, 0.857042304782, -0.188536767412, -0.291668520498, 0.343686800435, -0.399408727166, -0.179921593947, -0.010662340824, 0.23622232339, -0.257390617236, 0.213182376685, 0.576459373081, 0.306524087619, 0.248260630252, 0.050175145813, -0.101614469022, 0.281200318932, -0.391835525435, -0.0953767781907, -0.384721111012, 0.402854448732, -0.122585128952, 0.110591514785, -1.40623002748, 0.177662074116, -0.247743897977, -0.258200774495, -0.203357963386, 0.122486844237, 1.1677668027, -0.728672776285, 0.837006898515, 0.0198429228676, 0.209390034006]",0.0320841110315,500


# 4. Create input data for regression

This dataset contains housing prices.

In [7]:
%%sql
DROP TABLE IF EXISTS lin_housing;

CREATE TABLE lin_housing (id serial, 
                          x float8[], 
                          grp_by_col int, 
                          y float8);

INSERT INTO lin_housing VALUES
(1,ARRAY[0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98],1,24.00),
(2,ARRAY[0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14],1,21.60),
(3,ARRAY[0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03],1,34.70),
(4,ARRAY[0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94],1,33.40),
(5,ARRAY[0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33],1,36.20),
(6,ARRAY[0.02985,0.00,2.180,0,0.4580,6.4300,58.70,6.0622,3,222.0,18.70,394.12,5.21],1,28.70),
(7,ARRAY[0.08829,12.50,7.870,0,0.5240,6.0120,66.60,5.5605,5,311.0,15.20,395.60,12.43],1,22.90),
(8,ARRAY[0.14455,12.50,7.870,0,0.5240,6.1720,96.10,5.9505,5,311.0,15.20,396.90,19.15],1,27.10),
(9,ARRAY[0.21124,12.50,7.870,0,0.5240,5.6310,100.00,6.0821,5,311.0,15.20,386.63,29.93],1,16.50),
(10,ARRAY[0.17004,12.50,7.870,0,0.5240,6.0040,85.90,6.5921,5,311.0,15.20,386.71,17.10],1,18.90),
(11,ARRAY[0.22489,12.50,7.870,0,0.5240,6.3770,94.30,6.3467,5,311.0,15.20,392.52,20.45],1,15.00),
(12,ARRAY[0.11747,12.50,7.870,0,0.5240,6.0090,82.90,6.2267,5,311.0,15.20,396.90,13.27],1,18.90),
(13,ARRAY[0.09378,12.50,7.870,0,0.5240,5.8890,39.00,5.4509,5,311.0,15.20,390.50,15.71],1,21.70),
(14,ARRAY[0.62976,0.00,8.140,0,0.5380,5.9490,61.80,4.7075,4,307.0,21.00,396.90,8.26],1,20.40),
(15,ARRAY[0.63796,0.00,8.140,0,0.5380,6.0960,84.50,4.4619,4,307.0,21.00,380.02,10.26],1,18.20),
(16,ARRAY[0.62739,0.00,8.140,0,0.5380,5.8340,56.50,4.4986,4,307.0,21.00,395.62,8.47],1,19.90),
(17,ARRAY[1.05393,0.00,8.140,0,0.5380,5.9350,29.30,4.4986,4,307.0,21.00,386.85,6.58],1, 23.10),
(18,ARRAY[0.78420,0.00,8.140,0,0.5380,5.9900,81.70,4.2579,4,307.0,21.00,386.75,14.67],1,17.50),
(19,ARRAY[0.80271,0.00,8.140,0,0.5380,5.4560,36.60,3.7965,4,307.0,21.00,288.99,11.69],1,20.20),
(20,ARRAY[0.72580,0.00,8.140,0,0.5380,5.7270,69.50,3.7965,4,307.0,21.00,390.95,11.28],1,18.20);

SELECT * FROM lin_housing ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,x,grp_by_col,y
1,"[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98]",1,24.0
2,"[0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14]",1,21.6
3,"[0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03]",1,34.7
4,"[0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94]",1,33.4
5,"[0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]",1,36.2
6,"[0.02985, 0.0, 2.18, 0.0, 0.458, 6.43, 58.7, 6.0622, 3.0, 222.0, 18.7, 394.12, 5.21]",1,28.7
7,"[0.08829, 12.5, 7.87, 0.0, 0.524, 6.012, 66.6, 5.5605, 5.0, 311.0, 15.2, 395.6, 12.43]",1,22.9
8,"[0.14455, 12.5, 7.87, 0.0, 0.524, 6.172, 96.1, 5.9505, 5.0, 311.0, 15.2, 396.9, 19.15]",1,27.1
9,"[0.21124, 12.5, 7.87, 0.0, 0.524, 5.631, 100.0, 6.0821, 5.0, 311.0, 15.2, 386.63, 29.93]",1,16.5
10,"[0.17004, 12.5, 7.87, 0.0, 0.524, 6.004, 85.9, 6.5921, 5.0, 311.0, 15.2, 386.71, 17.1]",1,18.9


# 5. Regression

Now train a regression model using a multilayer perceptron with 2 hidden layers of 25 nodes each.

In [8]:
%%sql
DROP TABLE IF EXISTS mlp_regress;
DROP TABLE IF EXISTS mlp_regress_summary;
SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing',         -- Source table
    'mlp_regress',         -- Desination table
    'x',                   -- Input features
    'y',                   -- Dependent variable
    ARRAY[25,25],            -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',
    'relu',
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    TRUE              -- Verbose
);

Done.
Done.
1 rows affected.
1 rows affected.


mlp_regression
""


# 6.0 View the regression model

In [9]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([0.122856566817, -0.109511383377, 0.10922147183, 0.142820008065, 0.162756253894, -0.104436945183, 0.112036676092, 0.0467338442275, -0.16453372472, 0.0827992259598, 0.0137404038273, 0.0565353617298, -0.0246679212553, -0.042838429926, 0.381777598727, 0.276073801059, 0.00907788148497, 0.21191870498, -0.141702416989, 0.181936732347, -0.136061155165, -0.721381760449, -0.0560067549982, 0.203545531143, -0.00995912442451, 0.0613143120688, -0.0984596213056, -0.204436524678, 0.470767785122, -0.435311003078, -0.0424606276832, 0.160892203008, 0.0445352965619, -0.0879636030809, 0.154146037853, 0.0288780423027, -0.317282313525, 0.385922416222, -0.0664872317641, 0.146757511446, 0.133876976675, -0.116256543538, -0.0213680210249, 0.20015749221, -0.103457503483, -0.0425956948505, 0.121667463986, 0.204968623833, -0.159914333708, 0.149243857206, 0.0456901480449, -0.136975914168, -0.090899493726, 0.103816779262, 0.157303539757, -0.0813411072267, -0.186218530515, -0.18296662406, -0.0106273344335, -0.15268221771, -0.103478313334, 0.199984721908, 0.140135789025, 0.143966006971, -0.0970102681415, 0.0261086412469, -0.0330555393771, 0.0995076031524, 0.00130931383947, 0.0778756667869, -0.0527086470232, -0.600133053147, 0.0859133658584, 0.24113974288, 0.170332657512, 0.0676027349796, -0.21148456397, 0.248468295235, -0.436384189411, 0.175863820799, 0.123778486759, -0.282581458533, 0.0191701197018, 0.280887575877, 0.21926286976, -0.24666316945, -0.0262117477515, 0.268417193439, -0.0591642733853, 0.168335924156, 0.567262570173, -0.0244295268223, -0.161746761037, 0.239464436128, 0.0927106954455, 0.108399901337, -0.00684509495191, 0.0942018933997, 0.27681768162, -0.0200546475364, -0.0178865707443, 0.0590165895996, 0.178019822826, 0.270901895834, -0.180136838054, 0.201151867097, -0.269229552029, 0.162901821652, 0.176989876451, -0.0562545583224, 0.302891594425, -0.107643708081, 0.124875778612, -0.0717199493045, -0.256875036503, -0.104361508745, 0.0223166114387, -0.072220047336, 0.048066243185, 0.0917036420782, -0.0401521607798, -0.166103131, -0.0598667085507, 0.226655306977, 0.196020672052, -0.0019893331732, 0.298047303143, 0.0610271888562, -0.123756854539, -0.138425394609, -0.0813844369306, -0.211061219263, 0.280266006616, -0.456958897687, 0.0957847778583, 0.10211765487, -0.280257589355, 0.31403698235, -0.0988827746221, -0.315188453188, 0.0323558432208, 0.30938173242, -0.337991301037, -0.0746337727327, -0.128019139853, -0.107135213031, 0.401895277966, -0.0856407552996, 0.338660876519, 0.182709918475, 0.00468663392494, 0.296164404708, -0.149328993889, -0.455192142697, 0.123695517097, -0.101042506058, -0.299448111606, -0.121418328349, -0.015253490999, 0.105945563874, 0.206274179032, 0.0110203190195, -0.0198463256646, -0.122433280535, 0.159641322926, 0.0111190972641, 0.146024395139, -0.3292631924, 0.383620965193, -0.407475374511, 0.124656819426, 0.00814682875215, -0.196043525121, 0.259447828442, -0.0110058837001, -0.0911647683094, -0.495353629738, 0.158324486446, 0.284649054294, 0.159015240494, 0.127990452507, -0.0450611236751, 0.100307630204, -0.178426360702, 0.0795949564344, -0.0935834137435, 0.0121807964771, 0.13280000068, -0.0780024566724, 0.268183360375, -0.143863232988, 0.229637060781, 0.143828509532, 0.0467453366612, -0.101445117614, 0.110812332279, 0.274695568803, 0.0886704115438, 0.0920752574129, 0.230045146137, 0.189557642894, 0.175650286388, 0.612810225963, 0.445301543545, -0.115078741068, 0.157970457258, -0.0479338968094, 0.212561173909, 0.180880194985, -0.0870642554799, 0.142339596596, -0.151003784829, 0.0688697053283, -0.128056785862, 0.0793206934645, -0.118269324067, -0.0332473245304, -0.311866015062, -0.117922286012, 0.138892683591, -0.13204744327, 0.0179514948766, -0.019965660049, 0.132229635306, -0.0313989569283, -0.130315567213, 0.0845857500725, -0.159680095558, -0.0112748210112, -0.00992759022784, 0.164452997275, 0.0322838283196, -0.270242562524, 0.162183925006, -0.0574346808757, -0.22116233726, -0.047400722579, 0.00863094243228, 0.2113113507

# 7. Prediction for classification

In the following examples we will use the training data set for prediction as well, which is not usual but serves to show the syntax. First we will test the classification example. The prediction is in the the estimated_class_text column with the actual value in the class_text column.

In [10]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model',         -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,estimated_class_text,attributes,class_text,class
1,Iris-setosa,"[Decimal('5.1'), Decimal('3.5'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
2,Iris-setosa,"[Decimal('4.9'), Decimal('3.0'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
3,Iris-setosa,"[Decimal('4.7'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris-setosa,1
4,Iris-setosa,"[Decimal('4.6'), Decimal('3.1'), Decimal('1.5'), Decimal('0.2')]",Iris-setosa,1
5,Iris-setosa,"[Decimal('5.0'), Decimal('3.6'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
6,Iris-setosa,"[Decimal('5.4'), Decimal('3.9'), Decimal('1.7'), Decimal('0.4')]",Iris-setosa,1
7,Iris-setosa,"[Decimal('4.6'), Decimal('3.4'), Decimal('1.4'), Decimal('0.3')]",Iris-setosa,1
8,Iris-setosa,"[Decimal('5.0'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris-setosa,1
9,Iris-setosa,"[Decimal('4.4'), Decimal('2.9'), Decimal('1.4'), Decimal('0.2')]",Iris-setosa,1
10,Iris-setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris-setosa,1


Count the misclassifications

In [11]:
%%sql
SELECT COUNT(*) FROM mlp_prediction JOIN iris_data USING (id) 
WHERE mlp_prediction.estimated_class_text != iris_data.class_text;

1 rows affected.


count
0


# 8.  Prediction for regression

In [12]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress',               -- Model table
         'lin_housing',               -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT * FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,x,grp_by_col,y,estimated_y
1,"[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98]",1,24.0,23.997693578
2,"[0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14]",1,21.6,22.0225551504
3,"[0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03]",1,34.7,34.3269436787
4,"[0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94]",1,33.4,34.7421700033
5,"[0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]",1,36.2,35.1914922401
6,"[0.02985, 0.0, 2.18, 0.0, 0.458, 6.43, 58.7, 6.0622, 3.0, 222.0, 18.7, 394.12, 5.21]",1,28.7,29.5286073544
7,"[0.08829, 12.5, 7.87, 0.0, 0.524, 6.012, 66.6, 5.5605, 5.0, 311.0, 15.2, 395.6, 12.43]",1,22.9,23.2022360304
8,"[0.14455, 12.5, 7.87, 0.0, 0.524, 6.172, 96.1, 5.9505, 5.0, 311.0, 15.2, 396.9, 19.15]",1,27.1,23.364906529
9,"[0.21124, 12.5, 7.87, 0.0, 0.524, 5.631, 100.0, 6.0821, 5.0, 311.0, 15.2, 386.63, 29.93]",1,16.5,17.7779926867
10,"[0.17004, 12.5, 7.87, 0.0, 0.524, 6.004, 85.9, 6.5921, 5.0, 311.0, 15.2, 386.71, 17.1]",1,18.9,13.9266690258


In [15]:
%%sql
SELECT *, ABS(y-estimated_y) as abs_diff FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

20 rows affected.


id,x,grp_by_col,y,estimated_y,abs_diff
1,"[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98]",1,24.0,23.997693578,0.00230642201043
2,"[0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14]",1,21.6,22.0225551504,0.422555150371
3,"[0.02729, 0.0, 7.07, 0.0, 0.469, 7.185, 61.1, 4.9671, 2.0, 242.0, 17.8, 392.83, 4.03]",1,34.7,34.3269436787,0.373056321299
4,"[0.03237, 0.0, 2.18, 0.0, 0.458, 6.998, 45.8, 6.0622, 3.0, 222.0, 18.7, 394.63, 2.94]",1,33.4,34.7421700033,1.3421700033
5,"[0.06905, 0.0, 2.18, 0.0, 0.458, 7.147, 54.2, 6.0622, 3.0, 222.0, 18.7, 396.9, 5.33]",1,36.2,35.1914922401,1.00850775988
6,"[0.02985, 0.0, 2.18, 0.0, 0.458, 6.43, 58.7, 6.0622, 3.0, 222.0, 18.7, 394.12, 5.21]",1,28.7,29.5286073544,0.828607354372
7,"[0.08829, 12.5, 7.87, 0.0, 0.524, 6.012, 66.6, 5.5605, 5.0, 311.0, 15.2, 395.6, 12.43]",1,22.9,23.2022360304,0.302236030422
8,"[0.14455, 12.5, 7.87, 0.0, 0.524, 6.172, 96.1, 5.9505, 5.0, 311.0, 15.2, 396.9, 19.15]",1,27.1,23.364906529,3.735093471
9,"[0.21124, 12.5, 7.87, 0.0, 0.524, 5.631, 100.0, 6.0821, 5.0, 311.0, 15.2, 386.63, 29.93]",1,16.5,17.7779926867,1.27799268665
10,"[0.17004, 12.5, 7.87, 0.0, 0.524, 6.004, 85.9, 6.5921, 5.0, 311.0, 15.2, 386.71, 17.1]",1,18.9,13.9266690258,4.97333097422


In [16]:
%%sql
SELECT SQRT(SUM(ABS(y-estimated_y))/COUNT(y)) as rms_error FROM lin_housing JOIN mlp_regress_prediction USING (id);

1 rows affected.


rms_error
1.02862119016
